In [2]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
from sklearn.preprocessing  import LabelEncoder
import re

In [4]:
from sklearn.preprocessing import MinMaxScaler

In [7]:
data = pd.read_csv('training_titanic.csv')

In [8]:
data.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,2,"Weisz, Mrs. Leopold (Mathilde Francoise Pede)",female,29.0,1,0,228414,26.000,NaN,S,1
1,3,"Williams, Mr. Howard Hugh ""Harry""",male,NaN,0,0,A/5 2466,8.050,NaN,S,0
2,2,"Morley, Mr. Henry Samuel (""Mr Henry Marshall"")",male,39.0,0,0,250655,26.000,NaN,S,0
3,3,"Palsson, Mrs. Nils (Alma Cornelia Berglund)",female,29.0,0,4,349909,21.075,NaN,S,0
4,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.050,NaN,S,0


In [9]:
def nam(dataset):
    titles = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    # replace titles with a more common title or as Rare
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr',\
                                            'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    # convert titles into numbers
    dataset['Title'] = dataset['Title'].map(titles)
    # filling NaN with 0, to get safe
    dataset['Title'] = dataset['Title'].fillna(0)

In [10]:
def Ag(dataset):
    mean = dataset["Age"].mean()
    std = dataset["Age"].std()
    is_null = dataset["Age"].isnull().sum()
    # compute random numbers between the mean, std and is_null
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    # fill NaN values in Age column with random values generated
    age_slice = dataset["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    dataset["Age"] = age_slice
    dataset["Age"] = dataset["Age"].astype(int)

In [11]:
nam(data)

In [12]:
data.drop(columns = ['Name', 'Cabin', 'Ticket', 'Age', 'Pclass'], inplace=True)

In [13]:
data.head()

,Sex,SibSp,Parch,Fare,Embarked,Survived,Title
0,female,1,0,26.000,S,1,3
1,male,0,0,8.050,S,0,1
2,male,0,0,26.000,S,0,1
3,female,0,4,21.075,S,0,3
4,male,0,0,7.050,S,0,1


In [14]:
data.Embarked = data.Embarked.fillna(data.Embarked.mode()[0])

In [15]:
data.head()

,Sex,SibSp,Parch,Fare,Embarked,Survived,Title
0,female,1,0,26.000,S,1,3
1,male,0,0,8.050,S,0,1
2,male,0,0,26.000,S,0,1
3,female,0,4,21.075,S,0,3
4,male,0,0,7.050,S,0,1


In [16]:
data.head()

,Sex,SibSp,Parch,Fare,Embarked,Survived,Title
0,female,1,0,26.000,S,1,3
1,male,0,0,8.050,S,0,1
2,male,0,0,26.000,S,0,1
3,female,0,4,21.075,S,0,3
4,male,0,0,7.050,S,0,1


In [17]:
data = pd.get_dummies(data, columns=[ 'Sex', 'Title', "Embarked"], drop_first=True )

In [18]:
data.head()

,SibSp,Parch,Fare,Survived,Sex_male,Title_2,Title_3,Title_4,Title_5,Embarked_Q,Embarked_S
0,1,0,26.000,1,0,0,1,0,0,0,1
1,0,0,8.050,0,1,0,0,0,0,0,1
2,0,0,26.000,0,1,0,0,0,0,0,1
3,0,4,21.075,0,0,0,1,0,0,0,1
4,0,0,7.050,0,1,0,0,0,0,0,1


In [19]:
sc1 = MinMaxScaler()

In [21]:
X = sc1.fit_transform(data)

In [22]:
from sklearn.linear_model import LogisticRegression

In [23]:
features = data.columns.difference(["Survived"])
X = data[features]
y = data['Survived']

In [709]:
X

,Embarked_Q,Embarked_S,Fare,Parch,Sex_male,SibSp,Title_2,Title_3,Title_4,Title_5
0,0,1,26.0000,0,0,1,0,1,0,0
1,0,1,8.0500,0,1,0,0,0,0,0
2,0,1,26.0000,0,1,0,0,0,0,0
3,0,1,21.0750,4,0,0,0,1,0,0
4,0,1,7.0500,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
663,0,1,10.5000,0,0,0,1,0,0,0
664,1,0,7.7500,0,1,0,0,0,0,0
665,0,1,56.4958,0,1,0,0,0,0,0
666,0,1,9.8375,0,0,0,1,0,0,0


In [24]:
logmodel = LogisticRegression(max_iter=1000, C = 1.5, solver='liblinear')
logmodel.fit(X,y)

LogisticRegression(C=1.5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [25]:
logmodel.score(X, y)

0.8188622754491018

# Test data

In [26]:
test = pd.read_csv('test_titanic.csv')

In [27]:
nam(test)

In [28]:
test.drop(columns = ['Name', 'Cabin', 'Ticket', 'Age', 'Pclass'], inplace=True)

In [29]:
test.shape

(223, 6)

In [30]:
test.head()

,Sex,SibSp,Parch,Fare,Embarked,Title
0,male,1,1,36.7500,S,4
1,female,0,0,25.9292,S,5
2,male,0,0,7.7375,Q,1
3,female,2,1,27.0000,S,3
4,male,0,0,26.2875,S,1


In [31]:
test.Embarked = test.Embarked.fillna(test.Embarked.mode()[0])

In [32]:
test.Embarked.value_counts()

S    161
C     35
Q     27
Name: Embarked, dtype: int64

In [33]:
test = pd.get_dummies(test, columns=[ 'Sex', 'Title', "Embarked"], drop_first=True )

In [34]:
test.head()

,SibSp,Parch,Fare,Sex_male,Title_2,Title_3,Title_4,Title_5,Embarked_Q,Embarked_S
0,1,1,36.7500,1,0,0,1,0,0,1
1,0,0,25.9292,0,0,0,0,1,0,1
2,0,0,7.7375,1,0,0,0,0,1,0
3,2,1,27.0000,0,0,1,0,0,0,1
4,0,0,26.2875,1,0,0,0,0,0,1


In [35]:
test = test[['Embarked_Q', 'Embarked_S', 'Fare', 'Parch', 'Sex_male', 'SibSp',
       'Title_2', 'Title_3', 'Title_4', 'Title_5']]

In [36]:
pred = logmodel.predict(test)

In [37]:
np.savetxt('titanic_predictions_logistic_regression.csv',pred, fmt='%d')